In [2]:
import pandas as pd
import numpy as np
import relevanceeval

# Goal
This jupyter notebook serves as an area to test how to automatically test the results of our trained model to the expert annotations. It serves as a testing ground. 

In [3]:
# b_50k
sum(pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")["GitHubUrl"].isnull() == False)

14

In [4]:
# annotationStore

In [6]:
res_annot_merged = pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")

ans_in_res_DF = res_annot_merged[res_annot_merged["GitHubUrl"].isnull() == False][["Language","Query", "GitHubUrl", "Relevance", "Notes"]]
ans_in_res_DF.to_csv("./csv_output/rel_score_combined.csv")

In [7]:
ans_in_res_DF

,Language,Query,GitHubUrl,Relevance,Notes
365,Ruby,how to get html of website,https://github.com/sugaryourcoffee/syclink/blo...,0.0,NaN
387,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,1.0,NaN
388,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,2.0,NaN
389,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,1.0,NaN
469,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,2.0,NaN
470,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,2.0,"Don't ever use CBC mode (Bleichenbacher, etc.)"
471,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,3.0,NaN
472,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,3.0,NaN
1907,PHP,create cookie,https://github.com/cygnite/framework/blob/58d0...,1.0,NaN
2232,Python,copy to clipboard,https://github.com/spyder-ide/spyder/blob/f768...,1.0,The function performs pasting FROM the clipboa...


In [5]:
# Utilizing the relevanceeval.py
relevance_scores = relevanceeval.load_relevances("./Dataset/Testing/annotationStore.csv")

c:\Users\halom\Desktop\DSC180\Q1Project\DSC180Q1Project\relevanceeval.py:37: FutureWarning: The provided callable <function mean at 0x0000025145325D00> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  per_query_language = relevance_annotations.pivot_table(


In [6]:
# Loading in a prediction file generated by me.
predictions = relevanceeval.load_predictions("./csv_output/baseline_50k.csv")

In [7]:
predictions
relevance_scores

defaultdict(<function relevanceeval.load_relevances.<locals>.<lambda>()>,
            {'python': defaultdict(dict,
                         {'aes encryption': {'https://github.com/raymontag/kppy/blob/a43f1fff7d49da1da4b3d8628a1b3ebbaf47f43a/kppy/database.py#L861-L871': 2.5},
                          'copy to clipboard': {'https://github.com/spyder-ide/spyder/blob/f76836ce1b924bcc4efd3f74f2960d26a4e528e0/spyder/plugins/variableexplorer/widgets/collectionseditor.py#L1242-L1252': 2.3333333333333335},
                          'unzipping large files': {'https://github.com/OLC-Bioinformatics/sipprverse/blob/d4f10cdf8e1a39dac0953db61c21c97efc6006de/cgecore/utility.py#L471-L484': 1.3333333333333333}}),
             'java': defaultdict(dict,
                         {'copy to clipboard': {'https://github.com/percolate/caffeine/blob/e2265cab474a6397f4d75b1ed928c356a7b9672e/caffeine/src/main/java/com/percolate/caffeine/ClipboardUtils.java#L17-L27': 1.0}}),
             'php': defaultdict(dict,


In [8]:
print('% of URLs in predictions that exist in the annotation dataset:')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language])*100:.2f}%')

print('% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language], with_positive_relevance=True) * 100:.2f}%')

print('NDCG:')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.ndcg(predictions[language], relevance_scores[language]):.3f}')

% of URLs in predictions that exist in the annotation dataset:
	go: 0.00%
	java: 0.12%
	javascript: 0.00%
	php: 0.32%
	python: 0.20%
	ruby: 0.32%
% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):
	go: 0.00%
	java: 0.23%
	javascript: 0.00%
	php: 0.47%
	python: 0.23%
	ruby: 0.00%
NDCG:
	go: 0.000
	java: 0.000
	javascript: 0.000
	php: 0.001
	python: 0.004
	ruby: 0.000


In [9]:
# Results for baseline 20k NO FILTERING ANSWERS
# % of URLs in predictions that exist in the annotation dataset:
# 	go: 0.00%
# 	java: 0.25%
# 	javascript: 0.00%
# 	php: 0.32%
# 	python: 0.20%
# 	ruby: 0.64%
# % of URLs in predictions that exist in the annotation dataset (avg relevance > 0):
# 	go: 0.00%
# 	java: 0.23%
# 	javascript: 0.00%
# 	php: 0.47%
# 	python: 0.23%
# 	ruby: 0.00%
# NDCG:
# 	go: 0.000
# 	java: 0.006
# 	javascript: 0.000
# 	php: 0.001
# 	python: 0.005
# 	ruby: 0.000

In [10]:
#Newer working area

In [11]:
annotationStore = pd.read_csv("./Dataset/Testing/annotationStore.csv")
b_50k = pd.read_csv("./csv_output/baseline_50k_2.csv")

In [12]:
# b_50k
sum(pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")["GitHubUrl"].isnull() == False)

21

In [13]:
annotationStore

,Language,Query,GitHubUrl,Relevance,Notes
0,Go,unique elements,https://github.com/tylertreat/BoomFilters/blob...,0,NaN
1,Go,read properties file,https://github.com/gobs/httpclient/blob/a93d46...,0,NaN
2,Go,heatmap from 3d coordinates,https://github.com/twpayne/go-geom/blob/e21b3a...,1,NaN
3,Go,create cookie,https://github.com/volatiletech/abcweb/blob/9e...,2,NaN
4,Go,readonly array,https://github.com/ericchiang/k8s/blob/68fb216...,0,NaN
...,...,...,...,...,...
4001,Ruby,write csv,https://github.com/khiemns54/sp2db/blob/76c78d...,3,NaN
4002,Ruby,replace in file,https://github.com/gosu/ashton/blob/313673d8b1...,1,NaN
4003,Ruby,how to read the contents of a .gz compressed f...,https://github.com/lostisland/faraday_middlewa...,2,NaN
4004,Ruby,converting uint8 array to image,https://github.com/wvanbergen/chunky_png/blob/...,0,NaN


In [14]:
# annotationStore.drop_duplicates("GitHubUrl", keep="first").to_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")

In [15]:
relevance_annotations = pd.read_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")
per_query_language = relevance_annotations.pivot_table(
    index=['Query', 'Language', 'GitHubUrl'], values='Relevance', aggfunc=np.mean)

C:\Users\halom\AppData\Local\Temp\ipykernel_14424\2390078170.py:2: FutureWarning: The provided callable <function mean at 0x0000025145325D00> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  per_query_language = relevance_annotations.pivot_table(


In [16]:
# per_query_language
relevance_annotations["Relevance"].value_counts()

Relevance
0    1040
1     703
2     584
3     547
Name: count, dtype: int64

In [17]:
b_20k_test = pd.read_csv("./csv_output/baseline_20000k.csv")

In [18]:
b_20k_test["relevance"] = [0 for i in range(len(b_20k_test))]

In [19]:
annotationStore = pd.read_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")

In [20]:
b_20k_test

,Unnamed: 0,language,url,query,relevance
0,0,java,https://github.com/zaproxy/zaproxy/blob/0cbe51...,convert int to string,0
1,1,python,https://github.com/lacava/few/blob/5c72044425e...,convert int to string,0
2,2,php,https://github.com/s9e/RegexpBuilder/blob/59d0...,convert int to string,0
3,3,go,https://github.com/ricallinson/forgery/blob/e5...,convert int to string,0
4,4,java,https://github.com/samskivert/samskivert/blob/...,convert int to string,0
...,...,...,...,...,...
4679,4679,java,https://github.com/rjeschke/txtmark/blob/108ce...,how to read .csv file in an efficient way?,0
4680,4680,python,https://github.com/bcbio/bcbio-nextgen/blob/6a...,how to read .csv file in an efficient way?,0
4681,4681,python,https://github.com/jason-weirather/py-seq-tool...,how to read .csv file in an efficient way?,0
4682,4682,python,https://github.com/Azure/azure-kusto-python/bl...,how to read .csv file in an efficient way?,0


In [21]:
res_annot_merged = pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")
ans_in_res_DF = res_annot_merged[res_annot_merged["GitHubUrl"].isnull() == False]

In [22]:
res_annot_merged["Relevance"].fillna(0, inplace=True)#[res_annot_merged["url"].isnull() == False]

In [23]:
res_annot_merged["Relevance"].value_counts()

Relevance
0.0    4798
1.0       5
2.0       2
3.0       1
Name: count, dtype: int64

In [24]:
queries_with_annotations = list(ans_in_res_DF["query"].unique())

In [27]:
ndcgs = []
for qwa in queries_with_annotations:
    df_query = res_annot_merged[res_annot_merged["query"] == qwa]
    # print(len(df_query))
    # print(df_query["Relevance"].to_list())
    rel_lst = df_query["Relevance"].to_list()

    dcg = 0
    for i, val in enumerate(rel_lst):
        dcg += (val) / (np.log2(i + 2))

    # print(dcg)
    idcg = 0
    for i, val in enumerate(sorted(rel_lst, reverse=True)):
        idcg += (val) / (np.log2(i + 2))
    # print(idcg)
    ndcgs.append(dcg / idcg)
print(np.mean(ndcgs))

0.2882321334246341


In [63]:
res_annot_merged

,Unnamed: 0,language,url,query,Language,Query,GitHubUrl,Relevance,Notes
0,0,java,https://github.com/zaproxy/zaproxy/blob/0cbe51...,convert int to string,NaN,NaN,NaN,0.0,NaN
1,1,php,https://github.com/UnionOfRAD/lithium/blob/bec...,convert int to string,NaN,NaN,NaN,0.0,NaN
2,2,python,https://github.com/evhub/coconut/blob/ff971773...,convert int to string,NaN,NaN,NaN,0.0,NaN
3,3,python,https://github.com/lacava/few/blob/5c72044425e...,convert int to string,NaN,NaN,NaN,0.0,NaN
4,4,php,https://github.com/indeyets/pake/blob/22b4e1da...,convert int to string,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
4813,4801,python,https://github.com/wiheto/fetchopenfmri/blob/2...,how to read .csv file in an efficient way?,NaN,NaN,NaN,0.0,NaN
4814,4802,java,https://github.com/solita/clamav-java/blob/4ff...,how to read .csv file in an efficient way?,NaN,NaN,NaN,0.0,NaN
4815,4803,python,https://github.com/nicferrier/md/blob/302ca888...,how to read .csv file in an efficient way?,NaN,NaN,NaN,0.0,NaN
4816,4804,python,https://github.com/pandas-dev/pandas/blob/9feb...,how to read .csv file in an efficient way?,NaN,NaN,NaN,0.0,NaN


In [8]:
res_annot_merged = pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")

ans_in_res_DF = res_annot_merged[res_annot_merged["GitHubUrl"].isnull() == False]#[["Language","Query", "GitHubUrl", "Relevance", "Notes"]]


In [9]:
ans_in_res_DF.to_csv("./csv_output/rel_score_combined.csv")
ans_in_res_DF

,Unnamed: 0,language,url,query,Language,Query,GitHubUrl,Relevance,Notes
365,365,ruby,https://github.com/sugaryourcoffee/syclink/blo...,set working directory,Ruby,how to get html of website,https://github.com/sugaryourcoffee/syclink/blo...,0.0,NaN
387,387,python,https://github.com/OLC-Bioinformatics/sipprver...,set working directory,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,1.0,NaN
388,387,python,https://github.com/OLC-Bioinformatics/sipprver...,set working directory,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,2.0,NaN
389,387,python,https://github.com/OLC-Bioinformatics/sipprver...,set working directory,Python,unzipping large files,https://github.com/OLC-Bioinformatics/sipprver...,1.0,NaN
469,467,python,https://github.com/raymontag/kppy/blob/a43f1ff...,aes encryption,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,2.0,NaN
470,467,python,https://github.com/raymontag/kppy/blob/a43f1ff...,aes encryption,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,2.0,"Don't ever use CBC mode (Bleichenbacher, etc.)"
471,467,python,https://github.com/raymontag/kppy/blob/a43f1ff...,aes encryption,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,3.0,NaN
472,467,python,https://github.com/raymontag/kppy/blob/a43f1ff...,aes encryption,Python,aes encryption,https://github.com/raymontag/kppy/blob/a43f1ff...,3.0,NaN
1907,1902,php,https://github.com/cygnite/framework/blob/58d0...,create cookie,PHP,create cookie,https://github.com/cygnite/framework/blob/58d0...,1.0,NaN
2232,2227,python,https://github.com/spyder-ide/spyder/blob/f768...,copy to clipboard,Python,copy to clipboard,https://github.com/spyder-ide/spyder/blob/f768...,1.0,The function performs pasting FROM the clipboa...


In [10]:
relevance_scores = relevanceeval.load_relevances("./csv_output/rel_score_combined.csv")
predictions = relevanceeval.load_predictions("./csv_output/baseline_50k.csv")
languages_predicted = sorted(set(predictions.keys()))
languages_rs = sorted(set(relevance_scores.keys()))

c:\Users\halom\Desktop\DSC180\Q1Project\DSC180Q1Project\relevanceeval.py:37: FutureWarning: The provided callable <function mean at 0x0000016ECC2DBD80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  per_query_language = relevance_annotations.pivot_table(


In [11]:
predictions
relevance_scores

defaultdict(<function relevanceeval.load_relevances.<locals>.<lambda>()>,
            {'python': defaultdict(dict,
                         {'aes encryption': {'https://github.com/raymontag/kppy/blob/a43f1fff7d49da1da4b3d8628a1b3ebbaf47f43a/kppy/database.py#L861-L871': 2.5},
                          'copy to clipboard': {'https://github.com/spyder-ide/spyder/blob/f76836ce1b924bcc4efd3f74f2960d26a4e528e0/spyder/plugins/variableexplorer/widgets/collectionseditor.py#L1242-L1252': 2.3333333333333335},
                          'unzipping large files': {'https://github.com/OLC-Bioinformatics/sipprverse/blob/d4f10cdf8e1a39dac0953db61c21c97efc6006de/cgecore/utility.py#L471-L484': 1.3333333333333333}}),
             'java': defaultdict(dict,
                         {'copy to clipboard': {'https://github.com/percolate/caffeine/blob/e2265cab474a6397f4d75b1ed928c356a7b9672e/caffeine/src/main/java/com/percolate/caffeine/ClipboardUtils.java#L17-L27': 1.0}}),
             'php': defaultdict(dict,


In [12]:
print('% of URLs in predictions that exist in the annotation dataset:')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language])*100:.2f}%')

print('% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language], with_positive_relevance=True) * 100:.2f}%')

print('NDCG:')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.ndcg(predictions[language], relevance_scores[language]):.3f}')

% of URLs in predictions that exist in the annotation dataset:
	java: 100.00%
	php: 100.00%
	python: 66.67%
	ruby: 100.00%
% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):
	java: 100.00%
	php: 100.00%
	python: 66.67%
	ruby: 0.00%
NDCG:
	java: 1.000
	php: 1.000
	python: 0.667
	ruby: 0.000
